In [1]:
import torch
from torch.utils.data import DataLoader
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

from use import prediction
from get_datasets import get_datasets
from get_models import get_model
from tools import *

In [2]:
data_config = {}
data_config["path"] = 'datas/data_Foxwq_9d.csv'
data_config["data_size"] = 50000
data_config["offset"] = 0
data_config["data_type"] = "Word"
data_config["data_source"] = "foxwq" 
data_config["num_moves"] = 240
data_config["extend"] = False

In [3]:
_, testData = get_datasets(data_config, train=False)
test_loader = DataLoader(testData, batch_size=64, shuffle=False)

d:\codes\python\.vscode\Language_Go\get_datasets.py:152: DtypeWarning: Columns (347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_config["path"], encoding="ISO-8859-1", on_bad_lines='skip')


check_rate:0.96858
has 48429 games
transfer finish


steps finish


data finish


evalData shape:torch.Size([1162080, 240])
evalData memory size:2231193600


In [4]:
tgt = np.load('D://codes//python//.vscode//Language_Go//datas//acc30_diff.npy')
print(len(tgt))

72237


In [27]:
n = 10005
xw = testData.x[tgt[n]].cpu().numpy()
y = testData.y[tgt[n]].cpu().numpy()
xw = [transfer_back(x-1) for x in xw]
y = transfer_back(y)
print(xw)
print(y)

['pd', 'pp', 'dc', 'cp', 'cf', 'eq', 'qf', 'jc', 'lc', 'gc', 'qn', 'nq', 'hd', 'gd', 'jd', 'ic', 'id', 'fb', 'pk', 'oj', 'pj', 'oi', 'ok', 'mj', 'nk', 'lh', 'qq', 'qp', 'rp', 'pq', 'rr', 'qr', 'rq', 'kp', 'dk', 'kd', 'kc', 'ke', 'hf', 'gf', 'gg', 'he', 'if', 'fg', 'gh', 'md', 'nc', 'fh', 'gi', 'ef', 'cd', 'cg', 'bg', 'bf', 'ch', 'dg', 'be', 'bh', 'af', 'ci', 'kb', 'ej', 'hc', 'hb', 'ie', 'ja', 'ge', 'fe', 'ff', 'ed', 'kj', 'ji', 'jj', 'ii', 'hk', 'jg', 'fk', 'nd', 'oc', 'og', 'jp', 'jq', 'iq', 'ip', 'jo', 'hq', 'kq', 'ir', 'jr', 'qo', 'iq', 'pn', 'co', 'do', 'dn', 'qm', 'pr', 'or', 'qs', 'cn', 'bo', 'bp', 'dm', 'qg', 'pf', 'rh', 'rn', 'om', 'mk', 'jq', 'of', 'nf', 'pg', 'ph', 'ng', 'oh', 'rg', 'qh', 'ne', 'mf', 'iq', 'mm', 'jq', 'od', 'mc', 'qd', 'pc', 'rf', 're', 'sg', 'rd', 'bn', 'eo', 'dp', 'gq', 'gr', 'hp', 'hr', 'fp', 'km', 'jm', 'kl', 'ta', 'as', 'as', 'as', 'as', 'as', 'as', 'as', 'as', 'as', 'as', 'as', 'as', 'as', 'as', 'as', 'as', 'as', 'as', 'as', 'as', 'as', 'as', 'as', 'as

In [4]:
B5 = np.load('D://codes//python//.vscode//Language_Go//datas//B5_correct.npy')
R5 = np.load('D://codes//python//.vscode//Language_Go//datas//R5_correct.npy')
len(B5)
len(R5)


1162080

In [8]:
B5_moves = [i%240 for i in range(len(B5)) if B5[i]]
R5_moves = [i%240 for i in range(len(R5)) if R5[i]]

print(np.mean(B5_moves))
print(np.mean(R5_moves))

121.86532573454691
120.14697400502564


In [5]:
R_nB = [True  if R5[i] and not B5[i] else False for i in range(len(B5))]
R_nB_moves = [i%240 for i in range(len(B5)) if R_nB[i]]
print(np.mean(R_nB_moves))

105.57615352838694


In [6]:
nR_B = [True  if B5[i] and not R5[i] else False for i in range(len(B5))]
nR_B_moves = [i%240 for i in range(len(B5)) if R_nB[i]]
print(np.mean(nR_B_moves))

105.57615352838694


In [7]:
R_B = [True  if B5[i] and R5[i] else False for i in range(len(B5))]
R_B_moves = [i%240 for i in range(len(B5)) if R_nB[i]]
print(np.mean(R_B_moves))

105.57615352838694


In [8]:
nR_nB = [True if not B5[i] and not R5[i] else False for i in range(len(B5))]
nR_nB_moves = [i%240 for i in range(len(B5)) if R_nB[i]]
print(np.mean(nR_nB_moves))

105.57615352838694


In [9]:
import math
def distance(m1, m2):
    x1 = m1//19
    y1 = m1%19
    x2 = m2//19
    y2 = m2%19

    return math.sqrt(math.pow(x2-x1,2)+math.pow(y2-y1,2))

In [10]:
dis = []
trues = testData.y.cpu().numpy()
for i, check in enumerate(R_nB):
    if i%240:
        if check:
            dis.append(distance(trues[i], trues[i-1]))
    
        
print(np.mean(dis))

5.914109843663303


In [11]:
dis = []
trues = testData.y.cpu().numpy()
for i, check in enumerate(R_B):
    if i%240:
        if check:
            dis.append(distance(trues[i], trues[i-1]))
    
        
print(np.mean(dis))

2.0057476053307064


In [12]:
dis = []
trues = testData.y.cpu().numpy()
for i, check in enumerate(nR_B):
    if i%240:
        if check:
            dis.append(distance(trues[i], trues[i-1]))
    
        
print(np.mean(dis))

3.6618538315761215


In [13]:
dis = []
trues = testData.y.cpu().numpy()
for i, check in enumerate(nR_nB):
    if i%240:
        if check:
            dis.append(distance(trues[i], trues[i-1]))
    
        
print(np.mean(dis))

6.703647931122937


In [14]:
dis = []
trues = testData.y.cpu().numpy()
for i, check in enumerate(R_nB):
    if i%240:
        dis.append(distance(trues[i], trues[i-1]))

        
print(np.mean(dis))

3.069755614231316
